In [7]:
import json
from collections import Counter

# Пример словаря тональности
positive_words = {"хорошо", "отлично", "спасибо", "супер", "радость"}
negative_words = {"плохо", "ужасно", "проблема", "злость", "грусть"}

# Укажите полный путь к файлу
file_path = r'F:\PythonProject\MirrorMind\data\GN.json'

try:
    with open(file_path, 'r', encoding='utf-8') as f:
        chat_data = json.load(f)

    print("Файл успешно загружен!")
except FileNotFoundError:
    print(f"Файл не найден: {file_path}. Проверьте путь к файлу.")
# Анализ сообщений
messages = [msg["text"] for msg in chat_data["messages"] if "text" in msg and isinstance(msg["text"], str)]

# Подсчёт тональности
emotions = {"positive": 0, "negative": 0, "neutral": 0}
for message in messages:
    words = set(message.lower().split())
    if words & positive_words:
        emotions["positive"] += 1
    elif words & negative_words:
        emotions["negative"] += 1
    else:
        emotions["neutral"] += 1

# Вывод результатов
print("Эмоциональный фон чата:", emotions)


Файл успешно загружен!
Эмоциональный фон чата: {'positive': 83, 'negative': 30, 'neutral': 11678}


In [1]:
import json
from transformers import pipeline
import matplotlib.pyplot as plt
import pandas as pd

# 1. Подключение модели анализа тональности
sentiment_analyzer = pipeline("sentiment-analysis", model="blanchefort/rubert-base-cased-sentiment")

# 2. Загрузка данных чата из JSON-файла
file_path = 'F:\\PythonProject\\MirrorMind\\data\\GN.json'  # Замените на путь к вашему файлу
with open(file_path, 'r', encoding='utf-8') as f:
    chat_data = json.load(f)

# Извлечение текстов сообщений
messages = [msg["text"] for msg in chat_data["messages"] if "text" in msg and isinstance(msg["text"], str)]

# 3. Анализ тональности батчами
batch_size = 32  # Размер батча для обработки
results = []

for i in range(0, len(messages), batch_size):
    batch = messages[i:i + batch_size]
    batch_results = sentiment_analyzer(batch)
    results.extend(batch_results)

# 4. Подсчёт тональности
emotions = {"POSITIVE": 0, "NEGATIVE": 0, "NEUTRAL": 0}
for result in results:
    emotions[result['label']] += 1

print("Эмоциональный фон чата:")
print(emotions)

# 5. Визуализация данных
# Круговая диаграмма
labels = emotions.keys()
sizes = emotions.values()
colors = ['#8fd694', '#ff6b6b', '#d3d3d3']  # Цвета для POSITIVE, NEGATIVE, NEUTRAL
explode = (0.1, 0.1, 0)  # Выделяем позитивные и негативные

plt.figure(figsize=(8, 8))
plt.pie(sizes, labels=labels, colors=colors, explode=explode, autopct='%1.1f%%', startangle=140)
plt.title('Эмоциональный фон чата')
plt.show()

# Линейный график (если есть временные метки)
# Если у сообщений есть временные метки:
if "date" in chat_data["messages"][0]:
    timestamps = [msg["date"] for msg in chat_data["messages"] if "date" in msg]
    emotion_labels = [result['label'] for result in results]

    # Создание DataFrame
    data = {"timestamp": timestamps, "emotion": emotion_labels}
    df = pd.DataFrame(data)

    # Преобразование временных меток
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df.set_index('timestamp', inplace=True)

    # Группировка по дате
    time_emotions = df.groupby([pd.Grouper(freq='D'), 'emotion']).size().unstack(fill_value=0)

    # Построение графика
    time_emotions.plot(figsize=(10, 6), kind='line', title='Эмоциональный фон по времени')
    plt.xlabel('Дата')
    plt.ylabel('Количество сообщений')
    plt.show()


ModuleNotFoundError: No module named 'transformers'